In [0]:
%sql

-- 1. Create the Gold Table
-- We use CREATE OR REPLACE because we recalculate metrics daily
CREATE OR REPLACE TABLE gold_crypto_metrics AS
SELECT 
  coin_id,
  date,
  -- Get the closing price for the day (using the latest timestamp of that day)
  price_usd as close_price,
  
  -- Window Function: Calculate 7-Day Moving Average
  AVG(price_usd) OVER (
    PARTITION BY coin_id 
    ORDER BY date 
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) as moving_avg_7d,

  -- Window Function: Calculate Previous Day's Price to find growth
  LAG(price_usd) OVER (
    PARTITION BY coin_id 
    ORDER BY date
  ) as prev_day_price,

  -- Calculate Percentage Growth vs Yesterday
  ROUND(
    ((price_usd - LAG(price_usd) OVER (PARTITION BY coin_id ORDER BY date)) / 
    LAG(price_usd) OVER (PARTITION BY coin_id ORDER BY date)) * 100, 
  2) as daily_growth_pct

FROM silver_crypto_prices
-- We group by day to simplify (taking one record per day if multiple exist)
QUALIFY ROW_NUMBER() OVER (PARTITION BY coin_id, date ORDER BY event_time DESC) = 1;

-- 2. Check the results
SELECT * FROM gold_crypto_metrics 
WHERE coin_id = 'bitcoin' 
ORDER BY date DESC 
LIMIT 10;


Goal: Create a table ready for a dashboard. We will calculate a 7-Day Moving Average (to see trends) and Daily Volatility.